In [31]:
import cv2
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input


In [32]:
dataset_path = r"I:\Fall-25\CVPR_FINAL_ASSIGNMENT2_UPDATED\dataset"

X = []
Y = []
label_map = {}
current_label = 0

IMG_SIZE = 224  

for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_path):
        continue

    label_map[current_label] = person_name

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)

        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = preprocess_input(img)   # VGG19 preprocessing

        X.append(img)
        Y.append(current_label)

    current_label += 1

X = np.array(X)
Y = np.array(Y)

print("Classes:", label_map)
print("X shape:", X.shape)


Classes: {0: 'Dibbo', 1: 'Isti', 2: 'Maruf', 3: 'Yasin'}
X shape: (41, 224, 224, 3)


In [33]:
# Build model
model = keras.Sequential([
    data_augmentation,
    base_model,

    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),

    layers.Dense(num_classes, activation='softmax')
])



model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (8, 224, 224, 3)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (8, 7, 7, 1280)        │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (8, 7, 7, 1280)        │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (8, 1280)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (8, 256)               │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (8, 256)               │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (8, 256)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (8, 128)               │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (8, 128)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (8, 4)                 │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,417,063 (16.85 MB)

 Trainable params: 1,860,580 (7.10 MB)

 Non-trainable params: 2,556,483 (9.75 MB)

In [34]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    X,
    Y,
    epochs=11,
    batch_size=8,
    validation_split=0.2,
    shuffle=True
)



Epoch 1/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 944ms/step - accuracy: 0.5312 - loss: 1.2973 - val_accuracy: 0.1111 - val_loss: 1.6502
Epoch 2/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - accuracy: 0.5000 - loss: 1.2181 - val_accuracy: 0.2222 - val_loss: 1.6259
Epoch 3/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step - accuracy: 0.5625 - loss: 1.2299 - val_accuracy: 0.3333 - val_loss: 1.5962
Epoch 4/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step - accuracy: 0.7812 - loss: 0.5716 - val_accuracy: 0.3333 - val_loss: 1.5625
Epoch 5/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step - accuracy: 0.6875 - loss: 0.9395 - val_accuracy: 0.3333 - val_loss: 1.5229
Epoch 6/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step - accuracy: 0.8125 - loss: 0.6830 - val_accuracy: 0.3333 - val_loss: 1.4895
Epoch 7/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step - accuracy: 0.8750 - loss: 0.5259 - val_accuracy: 0.3333 - val_loss: 1.4610
Epoch 8/11
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step - accuracy: 0.8125 - loss: 0.4604 - val_accuracy: 0.3333 - val_loss

In [35]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

CONFIDENCE_THRESHOLD = 0.2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        pred = model.predict(face, verbose=0)
        confidence = np.max(pred)
        label = np.argmax(pred)

        if confidence < CONFIDENCE_THRESHOLD:
            name = "Unknown"
        else:
            name = label_map[label]

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(
            frame,
            f"{name} ({confidence:.2f})",
            (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0,255,0),
            2
        )

    cv2.imshow("EfficientNet Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
